In [185]:
# Let's load some other packages we need
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import itertools
import time
%matplotlib inline
import nibabel as nib # common way of importing nibabel

In [ ]:
# resting mri
mri_file = '/kaggle/input/d/genandlam/subj-1betas-session21/betas_session21.nii'
img = nib.load(mri_file)
print(type(img))

In [ ]:
def get_img(file_dir):
    img = nib.load(mri_file)
    return img

file=['/kaggle/input/d/genandlam/subj-1betas-session21/betas_session21.nii',
      '/kaggle/input/sub-1-betas-session22/sub_1_betas_session22.nii',
       '/kaggle/input/sub-2-betas-session21/sub_2_betas_session21.nii',
      '/kaggle/input/sub3-betas-session21/sub3_betas_session21.nii',
     '/kaggle/input/sub-4-betas-session21/sub_4_betas_session21.nii',
      '/kaggle/input/sub5-betas-session21/sub5_betas_session21.nii',
      '/kaggle/input/sub-6-betas-session/sub_6_betas_session21.nii',
      '/kaggle/input/sub-6-betas-session/sub_6_betas_session22.nii']

In [ ]:
def conv_np(img):
    img_data = img.get_fdata()
    print(type(img_data))  # it's a numpy array!
    print(img_data.shape)
    return img_data

In [ ]:
gender = pd.DataFrame({'subj': [1, 2,3,4,5,6,7,8], 'gender': ['M','M','F','F','F','F','M','M']})
gender

In [ ]:
from nilearn import plotting
from nilearn import image
mean_image = image.mean_img(img)
plotting.view_img(mean_image, threshold=None)

Coordinates of the slice we are interested in each direction. We will be using them for visualization.


In [ ]:
from nilearn.plotting import plot_epi

# cut in x-direction
sagittal = -25
# cut in y-direction
coronal = -37
# cut in z-direction
axial = -6

# coordinates displaying should be prepared as a list
cut_coords = [sagittal, coronal, axial]
plot_epi(mean_image, title="Smoothed mean EPI", cut_coords=cut_coords)

In [ ]:
# anatomical nifti
anat_mri_file = '/kaggle/input/subj-1-aseg/subj_1_aseg.nii'
anat_img = nib.load(anat_mri_file)
anat_img_data = anat_img.get_fdata()
print(type(anat_img_data))  # it's a numpy array!
print(anat_img_data.shape)

In [ ]:
from nilearn import datasets
msdl_atlas = datasets.fetch_atlas_msdl(data_dir='/kaggle/working')
msdl_coords = msdl_atlas.region_coords
n_regions = len(msdl_coords)

print(f'MSDL has {n_regions} ROIs, part of the following networks :\n{np.unique(msdl_atlas.networks)}.')

In [ ]:
from nilearn.maskers import NiftiLabelsMasker
from nilearn import maskers

# Instantiate the masker with label image and label values
masker = maskers.NiftiMapsMasker(
    msdl_atlas.maps,resampling_target="data",memory="nilearn_cache_",memory_level=1, detrend=True).fit()
# Visualize the atlas
# Note that we need to call fit prior to generating the mask


# At this point, no functional image has been provided to the masker.
# We can still generate a report which can be displayed in a Jupyter
# Notebook, opened in a browser using the .open_in_browser() method,
# or saved to a file using the .save_as_html(output_filepath) method.
#report = masker.generate_report()
#report

In [ ]:
roi_time_series = masker.transform(img)
roi_time_series.shape

In [ ]:
from nilearn.connectome import ConnectivityMeasure

correlation_measure = ConnectivityMeasure(kind='correlation')
correlation_matrix = correlation_measure.fit_transform([roi_time_series])[0]

In [ ]:
correlation_matrix.shape

In [ ]:
np.fill_diagonal(correlation_matrix, 0)
plotting.plot_matrix(correlation_matrix, labels=msdl_atlas.labels,
                     vmax=0.8, vmin=-0.8, colorbar=True)

In [ ]:
plotting.view_connectome(correlation_matrix, edge_threshold=0.2,
                         node_coords=msdl_atlas.region_coords)

In [ ]:
from nilearn.input_data import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure

data_dir = '/kaggle/working'
# load atlas
multiscale = datasets.fetch_atlas_basc_multiscale_2015(data_dir=data_dir)
atlas_filename = multiscale.scale064

# initialize masker (change verbosity)
masker = NiftiLabelsMasker(labels_img=atlas_filename, standardize=True,
                           memory='nilearn_cache', resampling_target="data",
                           detrend=True, verbose=0)

# initialize correlation measure, set to vectorize
correlation_measure = ConnectivityMeasure(kind='correlation', vectorize=True,
                                         discard_diagonal=True)

In [ ]:
all_features = [] # here is where we will put the data (a container)

for i,sub in enumerate(file):
    # extract the timeseries from the ROIs in the atlas
    img=get_img(sub)
    time_series = masker.fit_transform(img)
    # create a region x region correlation matrix
    correlation_matrix = correlation_measure.fit_transform([time_series])[0]
    # add to our container
    all_features.append(correlation_matrix)
    # keep track of status
    print('finished %s of %s'%(i+1,len(file)))

In [ ]:
all_features[0].shape

In [ ]:
# Let's save the data to disk
import numpy as np

np.savez_compressed('./MAIN_BASC064_subsamp_features', a=all_features)

In [ ]:
feat_file = './MAIN_BASC064_subsamp_features.npz'
X_features = np.load(feat_file)['a']
X_features.shape

In [ ]:
from sklearn.svm import SVC
l_svc = SVC(kernel='linear', class_weight='balanced') # define the model

l_svc.fit(X_features, gender['gender']) # fit the model

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, precision_score, f1_score

# predict the training data based on the model
y_pred = l_svc.predict(X_features)

# calculate the model accuracy
acc = l_svc.score(X_features,  gender['gender'])

# calculate the model precision, recall and f1, all in one convenient report!
cr = classification_report(y_true= gender['gender'],
                      y_pred = y_pred)

# get a table to help us break down these scores
cm = confusion_matrix(y_true= gender['gender'], y_pred = y_pred)

In [ ]:
import itertools
from pandas import DataFrame

# print results
print('accuracy:', acc)
print(cr)

# plot confusion matrix
cmdf = DataFrame(cm, index = ['M','F'], columns = ['M','F'])
sns.heatmap(cmdf, cmap = 'RdBu_r')
plt.xlabel('Predicted')
plt.ylabel('Observed')
# label cells in matrix
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j+0.5, i+0.5, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white")

In [ ]:
l_svc.coef_

In [ ]:
plt.bar(range(l_svc.coef_.shape[-1]),l_svc.coef_[0])
plt.title('feature importances')
plt.xlabel('feature')
plt.ylabel('weight')

In [ ]:
correlation_measure.inverse_transform(l_svc.coef_).shape

In [ ]:
from nilearn import plotting

feat_exp_matrix = correlation_measure.inverse_transform(l_svc.coef_)[0]

plotting.plot_matrix(feat_exp_matrix, figure=(10, 8),  
                     labels=range(feat_exp_matrix.shape[0]),
                     reorder=False,
                    tri='lower')

In [ ]:
coords = plotting.find_parcellation_cut_coords(atlas_filename)

In [ ]:
plotting.plot_connectome(feat_exp_matrix, coords, colorbar=True)

In [ ]:
plotting.plot_connectome(feat_exp_matrix, coords, colorbar=True, edge_threshold=0.005)

In [ ]:
plotting.view_connectome(feat_exp_matrix, coords, edge_threshold='60%')